In [32]:
import pandas as pd
import sklearn
f = pd.read_csv("daily_weather_raw.csv", sep=',', parse_dates=True, infer_datetime_format=True)
f.head(1)

,date,air_pressure_DAYMAX,air_temp_DAYMAX,avg_wind_speed_DAYMAX,max_wind_speed_DAYMAX,rain_accumulation_DAYMAX,rain_duration_DAYMAX,relative_humidity_DAYMAX,air_pressure_DAYMIN,air_temp_DAYMIN,...,relative_humidity_9am,air_pressure_3pm,air_temp_3pm,avg_wind_direction_3pm,avg_wind_speed_3pm,max_wind_direction_3pm,max_wind_speed_3pm,rain_accumulation_3pm,rain_duration_3pm,relative_humidity_3pm
0,2011-09-10,914.9,68.0,8.9,9.3,0.0,0.0,90.8,911.1,52.7,...,86.27,913.05,61.394,211.9,5.83,219.0,6.92,0.0,0.0,69.9


In [33]:
f[['relative_humidity_DAYMAX','relative_humidity_9am','relative_humidity_DAYMIN','relative_humidity_3pm']].head(1)

,relative_humidity_DAYMAX,relative_humidity_9am,relative_humidity_DAYMIN,relative_humidity_3pm
0,90.8,86.27,29.0,69.9


In [34]:
def functionG(row):
    if row['relative_humidity_3pm'] < 25:
        val = 1
    elif row['relative_humidity_3pm'] > 25:
        val = 0
    else:
        val = 0
    return val

In [35]:
f['label']=f.apply(functionG, axis=1)

In [36]:
f.shape

(1097, 34)

In [37]:
#REMOVE HUMIDITY
cols = ['relative_humidity_DAYMAX','relative_humidity_DAYMIN']

In [38]:
for c in cols:
    del f[c]
f.shape

(1097, 32)

In [39]:
#REMOVE DAY features
cols = [c for c in f.columns if ('day' in c.lower())]
for c in cols:
    del f[c]
f.shape

(1097, 20)

In [40]:
#REMOVE 3pm features
cols = [c for c in f.columns if ('3pm' in c.lower())]
cols.remove('relative_humidity_3pm')

In [41]:
for c in cols:
    del f[c]
f.shape

(1097, 12)

In [42]:
f.columns

Index([u'date', u'air_pressure_9am', u'air_temp_9am',
       u'avg_wind_direction_9am', u'avg_wind_speed_9am',
       u'max_wind_direction_9am', u'max_wind_speed_9am',
       u'rain_accumulation_9am', u'rain_duration_9am',
       u'relative_humidity_9am', u'relative_humidity_3pm', u'label'],
      dtype='object')

In [43]:
# CONVERT wind from meter per sec to miles per hour

f['avg_wind_speed_9am'] = f['avg_wind_speed_9am'].apply(lambda x: x*2.23694)
f['max_wind_speed_9am'] = f['max_wind_speed_9am'].apply(lambda x: x*2.23694)

In [44]:
p = f.dropna()
f.shape, p.shape

((1097, 12), (1074, 12))

In [45]:
label1 = f.dropna().loc[f['label'] == 1]
label0 = f.dropna().loc[f['label'] == 0]
label0.shape, label1.shape

((876, 12), (198, 12))

In [46]:
label0 =label0.sample(547, random_state=42)

f=label0.append(label1)
f.reset_index(inplace=True)
del f['index']
ls

f.shape

(745, 12)

In [47]:
#from imblearn.over_sampling import SMOTE
#smote = SMOTE(ratio = 'auto', kind = 'regular', random_state=12)

lastcolumn = f.shape[1]

features = f.columns[1:lastcolumn-1]
target = f.columns[lastcolumn-1]

x=f[features].copy(deep=True)
y=f[target].copy(deep=True)

#X, Y = smote.fit_sample(x, y)

In [48]:
X = pd.DataFrame(x)
Y = pd.DataFrame(y)
X.columns, Y.columns

(Index([u'air_pressure_9am', u'air_temp_9am', u'avg_wind_direction_9am',
        u'avg_wind_speed_9am', u'max_wind_direction_9am', u'max_wind_speed_9am',
        u'rain_accumulation_9am', u'rain_duration_9am',
        u'relative_humidity_9am', u'relative_humidity_3pm'],
       dtype='object'), Index([u'label'], dtype='object'))

In [50]:
X.columns = features
Y.columns = ['label'] 

In [51]:
X.columns, Y.columns

(Index(['air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
        'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
        'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
        'relative_humidity_3pm'],
       dtype='object'), Index(['label'], dtype='object'))

In [52]:
# Store a copy to write to CSV 
write_to_csv = X.copy(deep=True)
write_to_csv.columns

Index(['air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
       'relative_humidity_3pm'],
      dtype='object')

# Classification

In [53]:
del X['relative_humidity_3pm']
del X['relative_humidity_9am']
X.columns

Index(['air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am'],
      dtype='object')

In [54]:
Y.shape, Y.sum()

((1094, 1), label    547
 dtype: int64)

In [60]:
########################
# RANDOM SPLIT
########################

from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.30, random_state=30)

In [61]:
Y.sum(), Y_test.sum(), Y_train.sum()

(label    547
 dtype: int64, label    172
 dtype: int64, label    375
 dtype: int64)

## Decision Tree Classifier

In [62]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, Y_train)
Z = clf.predict(X_test)

In [63]:
Z.sum(), Y_test.sum(), Y_test.shape[0]-Y_test.sum()

(177, label    172
 dtype: int64, label    157
 dtype: int64)

In [64]:
from sklearn.metrics import classification_report
# 1 means LOW humidity
print(classification_report(Y_test, Z))

             precision    recall  f1-score   support

          0       0.84      0.81      0.82       157
          1       0.83      0.85      0.84       172

avg / total       0.83      0.83      0.83       329



In [65]:
sklearn.metrics.accuracy_score(Y_test, Z, normalize = True)

0.83282674772036469

## Naive Bayes

In [66]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb_classifier = gnb.fit(X_train, Y_train)
Z_NaiveBayes = gnb_classifier.predict(X_test)

/data/home/a1singh/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [67]:
# 1 means LOW humidity
print(classification_report(Y_test, Z_NaiveBayes))

             precision    recall  f1-score   support

          0       0.97      0.19      0.32       157
          1       0.57      0.99      0.73       172

avg / total       0.76      0.61      0.53       329



In [68]:
sklearn.metrics.accuracy_score(Y_test, Z_NaiveBayes, normalize = True)

0.61094224924012153

# Writing data to csv

In [69]:
df = write_to_csv.copy(deep=True)
df.shape

(1094, 10)

In [70]:
# shuffle
import numpy as np
df = df.iloc[np.random.permutation(len(df))]
df = df.reset_index(drop=True)

In [71]:
import random

ix = [(row, col) for row in range(df.shape[0]) for col in range(df.shape[1]-1)]

for row, col in random.sample(ix, int(31)):
    df.iat[row, col] = np.nan

In [72]:
df.to_csv("daily_weather.csv")

In [76]:
f2 = pd.read_csv("daily_weather.csv", sep=',', parse_dates=True, infer_datetime_format=True, index_col=0)
f2.shape, X.shapeipython

((1094, 10), (745, 10))

In [77]:
f2.loc[len(f2)]= X.iloc[555]

In [78]:
f2.shape

(1095, 10)

In [79]:
f2.to_csv("daily_weather.csv")